5<a href="https://colab.research.google.com/github/NatnaelTDaba/Object-Detection/blob/master/xView_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
work_dir = '/content/drive/MyDrive/UCF/CAP7919/Object-Detection/'
data_dir = work_dir+'data/'
util_dir = work_dir+'data_utilities/'
os.chdir(work_dir)
sys.path.append(util_dir)
sys.path.append(data_dir)

In [3]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
import numpy as np
import csv
%matplotlib inline
from PIL import Image, ImageDraw # to read images
import cv2
from tqdm import tqdm
import pickle

In [4]:
#Loading our labels
coords, chips, classes = wv.get_labels(data_dir+'xView_train.geojson')

100%|██████████| 601937/601937 [00:03<00:00, 163158.17it/s]


In [5]:
labels = {}
with open(util_dir+'xview_class_labels.txt') as f:
    for row in csv.reader(f):
        labels[int(row[0].split(":")[0])] = row[0].split(":")[1]

valid_labels = list(labels.keys())

In [ ]:
open_file = open("valid_chip_names.pkl", "rb")
valid_chip_names = pickle.load(open_file)
open_file.close()

In [ ]:
len(valid_chip_names)

846

In [ ]:
unique, counts = np.unique(classes, return_counts=True)
class_counts = {k:0 for k in list(unique.astype('int64'))}

In [ ]:
org_data_dir = data_dir+'train_images/'
tgt_data_dir = data_dir+'new_train_images/'
chip_names = list(np.unique(chips))
k = 0

imgs_processed = 0

with tqdm(total=len(chip_names), position=0, leave=True) as pbar:

  for chip_name in tqdm(chip_names, position=0, leave=True):

    if chip_name in valid_chip_names:

      chip = np.array(Image.open(org_data_dir+chip_name))

      chip_bboxes = coords[chips==chip_name]
      bbox_classes = classes[chips==chip_name].astype(np.int64)
      imgs_processed += 1
      for i in range(len(chip_bboxes)):

        cropped_img = chip[int(chip_bboxes[i][1])-10:int(chip_bboxes[i][3])+10, int(chip_bboxes[i][0])-10:int(chip_bboxes[i][2])+10]

        if cropped_img.shape[0] > 0 and cropped_img.shape[1] > 0:

          im = Image.fromarray(cropped_img)
          im.save(tgt_data_dir+str(bbox_classes[i])+'/'+str(k+1)+'.jpg')
          k += 1
          class_counts[bbox_classes[i]] += 1

    pbar.update()

print("\nNumber of extracted images:", k)
print("Number of chips processed:", imgs_processed)

100%|██████████| 847/847 [1:20:22<00:00,  5.69s/it]


Number of extracted images: 593162
Number of chips processed: 846


In [ ]:
new_pixel_areas = []
old_pixel_areas = []
low_thresh = 6
high_thresh = 10
num_removed_images = 0
with tqdm(total=60, position=0, leave=True) as pbar:
  for subdir, dirs, files in tqdm(os.walk(data_dir+'new_train_images/'), position=0, leave=True):
      for filename in files:
        filepath = subdir + os.sep + filename
        if filepath.endswith(".jpg"):
          img = np.array(Image.open(filepath))
          area = img.shape[0]*img.shape[1]
          old_pixel_areas.append(area)
          if (np.log(area) > high_thresh) or (np.log(area) < low_thresh):
            os.remove(filepath)
            num_removed_images+=1
          else:
            new_pixel_areas.append(area)

      pbar.update()

63it [19:53, 18.95s/it]
63it [19:53, 18.95s/it]


In [6]:
total_num_of_images = 0
class_counts_new = {}
for subdir, dirs, files in os.walk(data_dir+'new_train_images'):
  print(subdir[-2:],"len:",len(files))
  if len(files)>0:
    class_counts_new[int(subdir[-2:])] = len(files)
  total_num_of_images+=len(files)
print(total_num_of_images)

es len: 0
11 len: 45
12 len: 166
13 len: 199
15 len: 37
17 len: 759
19 len: 2967
20 len: 331
21 len: 1548
23 len: 5315
24 len: 2373
25 len: 1253
26 len: 226
27 len: 1757
28 len: 394
29 len: 69
32 len: 64
33 len: 4
34 len: 1492
35 len: 649
36 len: 42
37 len: 63
38 len: 93
40 len: 366
41 len: 682
42 len: 519
44 len: 93
45 len: 53
47 len: 539
49 len: 90
50 len: 121
51 len: 31
52 len: 9
53 len: 79
54 len: 40
55 len: 61
56 len: 18
57 len: 3
59 len: 125
60 len: 648
61 len: 46
62 len: 50
63 len: 250
64 len: 503
65 len: 89
66 len: 39
71 len: 258
72 len: 408
74 len: 41
75 len: 9
76 len: 361
77 len: 312
79 len: 818
82 len: 15
83 len: 1020
84 len: 23
86 len: 701
89 len: 281
91 len: 250
93 len: 24
94 len: 48
28869


In [ ]:
!mkdir ../backup_new_train_images_2/

In [ ]:
!cp -r ../new_train_images_2/ ../backup_new_train_images_2/

In [ ]:
!rm -r ../backup_new_train_images_2/

^C


In [ ]:
!mkdir ../new_train_images_2

mkdir: cannot create directory ‘../new_train_images_2’: File exists


In [ ]:
for i in list(unique.astype('int64')):
  os.mkdir('../new_train_images_2/'+str(i))

In [ ]:
!mkdir data/new_train_images

In [ ]:
for i in list(unique):
  os.mkdir(data_dir+'new_train_images/'+str(int(i)))